# 30-121 : Tidy up the Hyperparameter code

## Web References

- [Getting started with KerasTuner](https://keras.io/guides/keras_tuner/getting_started/)
- [Hyper parameter optimisation with Hyperopt and MLflow](https://www.youtube.com/watch?v=W_mJxQupJ4I)
- [hyperopt_logistic_regression.ipynb](https://github.com/amoat7/mlflow_tutorial/blob/master/notebooks/hyperopt_logistic_regression.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
from functools import partial

import pandas as pd
import tensorflow as tf
import keras as k
import kerastuner as kt

from competition import source_data as sd
from competition import data_preparation as dp
from competition import feature_engineering as fe
from competition import model_data as md
from competition import model_training as mt
from competition import model_layers as ml
from competition import model_definitions as mm

import mlflow
import mlflow.keras

<ipython-input-2-b0e74d1bd85b>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
# Set the GPU memory for growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Configure Logging

In [4]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2023-03-26 12:09:09 INFO     Started


## Data Collection

In [5]:
# load the source training set
df_source = sd.read_csv('../data/train.csv.gz',
                        compression='gzip',
                        dtype=sd.source_dtype)

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [6]:
# load the source training labels
df_source_labels = sd.read_csv('../data/train_labels.csv')

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Data Preparation & Cleaning

In [7]:
# prepare the main dataset
df_source = dp.prepare_main_dataset(df_source,
                                    elapsed_time_min_clip=0,
                                    elapsed_time_max_clip=3691298)

# remove sessions with problems
problem_sessions = dp.find_problem_sessions(df_source)
df_source = df_source[~df_source['session_id'].isin(problem_sessions)]

In [8]:
# prepare the label dataset
df_source_labels = dp.prepare_label_dataset(df_source_labels)

# remove sessions with problems
df_source_labels = df_source_labels[~df_source_labels['session_id'].isin(problem_sessions)]

## Feature Engineering

In [9]:
# create the initial features
df_features = fe.create_initial_features(df_source, df_source_labels)

In [10]:
# add the feature to the features dataset
df_features = fe.add_elapsed_time_features(
    features=df_features,
    X=df_source)

In [11]:
# add the total count features to the features dataset
df_features = fe.add_count_total_features(
    features=df_features,
    X=df_source)

In [12]:
# add the unique count features to the features dataset
df_features = fe.add_count_unique_features(
    features=df_features,
    X=df_source)

In [13]:
# # add the screen heatmap feature to the features dataset
# df_features = fe.add_screen_heatmap_feature(df_features, df_source)

# with pd.option_context('display.max_columns', None):
#     display(df_features.head(6))

## Data Selection

In [14]:
random_state = 51

In [15]:

# split the dataset into train, validation and test sets
train, val, test = md.select_sessions(
    y=df_source_labels.head(100),
    random_state=random_state,
    test_size=0.60,
    train_size=0.75)

Train: 30
Validation: 10
Test: 60


## Model Training

### Create Datasets

In [16]:
# set the feature list
feature_list = ['elapsed_time_sum', 'elapsed_time_max', 'elapsed_time_min', 'elapsed_time_mean', 'elapsed_time_mode']

# create the simple model dataset
simple_model_dataset = md.get_feature_dataset(
    features=df_features,
    y=df_source_labels,
    feature_list=feature_list,
    train=train,
    val=val,
    test=test,
    include_question=True,
    expand_question=False)

2023-03-26 12:09:56 INFO     -- Creating the train dataset
2023-03-26 12:09:56 INFO     Creating the dataset for all level groups


  0%|          | 0/540 [00:00<?, ?it/s]

2023-03-26 12:09:57 INFO     -- Creating the val dataset
2023-03-26 12:09:57 INFO     Creating the dataset for all level groups


  0%|          | 0/180 [00:00<?, ?it/s]

2023-03-26 12:09:57 INFO     -- Creating the test dataset
2023-03-26 12:09:57 INFO     Creating the dataset for all level groups


  0%|          | 0/1080 [00:00<?, ?it/s]

In [17]:
# define the simple model input shape
input_data = simple_model_dataset['train']['X']
simple_model_shape = input_data.shape[1]
print('simple_model_shape:', simple_model_shape)

# define the output shape
output_data = simple_model_dataset['train']['y']
simple_model_output_shape = 1
print('output_shape', simple_model_output_shape)

simple_model_shape: 23
output_shape 1


### Training

In [18]:
# create the experiment
mlflow.set_experiment("Hyper Parameter Tuning")


<Experiment: artifact_location='file:///workspaces/dsm150-2022-oct/cw02/phase_03/mlruns/783357371002213942', creation_time=1679784859585, experiment_id='783357371002213942', last_update_time=1679784859585, lifecycle_stage='active', name='Hyper Parameter Tuning', tags={}>

#### Simple Dense

In [44]:
mlflow.end_run()

In [48]:
# create the hyperparameter object
hp = kt.HyperParameters()

model = mm.tune_simple_dense_model(
    dataset=simple_model_dataset,
    max_trials=10,
    input_shape=simple_model_shape,
    output_shape=simple_model_output_shape,
    dense_layer_count=hp.Int('dense_layer_count', min_value=1, max_value=3, step=1),
    dense_units=hp.Int('dense_units', min_value=32, max_value=512, step=32),
    dense_activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']),
    dense_l1_regulization=hp.Float('dense_l1_regulization', min_value=0.0, max_value=0.1, step=0.01),
    dense_l2_regulization=hp.Float('dense_l2_regulization', min_value=0.0, max_value=0.1, step=0.01),
    dense_dropout=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05),
    train_epochs=10,
    train_batch_size=10,
    train_optimizer=k.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
    train_loss='binary_crossentropy',
    train_metrics=['accuracy'],
    train_class_weight=None)

model.summary()


Trial 1 Complete [00h 00m 05s]
val_accuracy: 0.7888888716697693

Best val_accuracy So Far: 0.7888888716697693
Total elapsed time: 00h 00m 05s
INFO:tensorflow:Oracle triggered exit
2023-03-26 13:10:27 INFO     Oracle triggered exit
{}
2023-03-26 13:10:27 INFO     Creating simple dense model
6/6 [==============================] - 0s 1ms/step - loss: 0.4476 - accuracy: 0.7889
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 23)]              0         
                                                                 
 dense (Dense)               (None, 32)                768       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________

In [ ]:
# # define the hyperparameter tuning function
# def build_simple_model(hp):
#     global simple_model_shape
#     global simple_model_output_shape

#     # create the model
#     model = mm.get_simple_dense_model(
#         input_shape=simple_model_shape,
#         output_shape=simple_model_output_shape,
#         dense_layer_count=hp.Int('dense_layer_count', min_value=1, max_value=3, step=1),
#         dense_units=hp.Int('dense_units', min_value=32, max_value=512, step=32),
#         dense_activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']),
#         dense_l1_regulization=hp.Float('dense_l1_regulization', min_value=0.0, max_value=0.1, step=0.01),
#         dense_l2_regulization=hp.Float('dense_l2_regulization', min_value=0.0, max_value=0.1, step=0.01),
#         dense_dropout=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05),
#         compile=True,
#         optimizer=k.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
#         loss='binary_crossentropy',
#         metrics=['accuracy'])

#     return model

# # define CustomSearch class
# class CustomSearch(kt.tuners.RandomSearch):
#     # def pre_create_trial(self):
#     #     logging.info('pre_create_trial')
#     #     mlflow.start_run(nested=True)
#     #     super(CustomSearch, self).pre_create_trial()

#     def on_trial_begin(self, trial):
#         logging.info('on_trial_begin')
#         #if int(trial.trial_id) > 0:
#         mlflow.keras.autolog()
#         mlflow.start_run(nested=True)
#             #mlflow.keras.autolog(disable=False)

#         super(CustomSearch, self).on_trial_begin(trial)

#     def on_trial_end(self, trial):
#         logging.info('on_trial_end')
#         mlflow.end_run()

#         super(CustomSearch, self).on_trial_end(trial)

# # create the tuner
# with mlflow.start_run() as run:
#     tuner = CustomSearch(
#         build_simple_model,
#         objective="val_accuracy",
#         max_trials=10,
#         executions_per_trial=1,
#         overwrite=True)
    
#     run_id = run.info.run_id
# mlflow.end_run()
# mlflow.delete_run(run_id)

# with mlflow.start_run():
#     mlflow.keras.autolog()
#     #mlflow.start_run(nested=True)

#     # search for the best hyperparameters
#     tuner.search(
#         simple_model_dataset['train']['X'],
#         simple_model_dataset['train']['y'],
#         validation_data=(simple_model_dataset['val']['X'], simple_model_dataset['val']['y']),
#         epochs=10,
#         callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])
    
#     #mlflow.end_run()

#     # log the best hyperparameters
#     best_hp = tuner.get_best_hyperparameters()[0].values
#     mlflow.log_params(best_hp)
#     print(best_hp)

#     # Retrieve the best model, evaluate it, and log the metrics
#     best_model = tuner.get_best_models()[0]
#     val_loss, val_accuracy = best_model.evaluate(simple_model_dataset['val']['X'], simple_model_dataset['val']['y'])
#     mlflow.log_metric("val_loss", val_loss)
#     mlflow.log_metric("val_accuracy", val_accuracy)

# mlflow.end_run()

In [ ]:
# # log the parameters
# mlflow.set_tag('model', 'simple-dense-model')
# mlflow.set_tag('notebook', '03-101_mlflow.ipynb')
# mt.log_params(simple_model_dataset, feature_list, random_state)

# # train the model
# model = mm.train_simple_dense(
#     dataset=simple_model_dataset,
#     input_shape=simple_model_shape,
#     output_shape=simple_model_output_shape,
#     dense_layer_count=2,
#     dense_units=1024,
#     dense_activation='relu',
#     dense_l1_regulization=1e-5,
#     dense_l2_regulization=0.0,
#     dense_dropout=0.0,
#     train_epochs=1000,
#     train_batch_size=1000,
#     train_optimizer='adam',
#     train_loss='binary_crossentropy',
#     train_metrics=['accuracy'],
#     train_class_weight=None)

# mlflow.end_run()